In [ ]:
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import glob
import os
import posixpath

import torch
from torchvision.transforms import v2
import torchvision

import numpy as np
import cv2 as cv
import math
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d
from scipy import interpolate
from numpy import asarray
import torchvision.transforms as transforms
import statistics


In [ ]:
source1 = sorted(glob.glob('./datasample/test_set_yolo/*_label.jpg'), key = lambda i: (len(posixpath.basename(i)), i))
source2 = sorted(glob.glob('./output/processed/*_prediction.jpg'), key = lambda i: (len(posixpath.basename(i)), i))
source3 = sorted(glob.glob('./datasample/test_set_yolo/test*.jpg'), key = lambda i: (len(posixpath.basename(i)), i))


# Post Processing
Brings Unet predictions back to original size

In [ ]:
from pathlib import Path
Path("./output/rmse/").mkdir(parents=True, exist_ok=True)
Path("./output/spline/").mkdir(parents=True, exist_ok=True)

a = []
c = [] 
for label, prediction, image in zip(source1, source2, source3):
    spine = cv.imread(image)
    spine_pred = cv.imread(image)
    im_label = cv.imread(label)
    im_pred = cv.imread(prediction)
    
    im_o = cv.cvtColor(im_label, cv.COLOR_BGR2GRAY)
    im_p = cv.cvtColor(im_pred, cv.COLOR_BGR2GRAY)

    ret1, thresh1 = cv.threshold(im_o, 127, 255, 0)
    contours1, hierarchy1 = cv.findContours(thresh1, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_TC89_L1 )
 
    ret2, thresh2 = cv.threshold(im_p, 127, 255, 0)
    contours2, hierarchy2 = cv.findContours(thresh2, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_TC89_L1 )

    k = []
    less_contours1 = []
    for cnt1 in contours1: 
          less_contours1.append(cnt1)
          k.append(cv.contourArea(cnt1))
    
    s=[]
    for cn in contours2:
        s.append(cv.contourArea(cn))
        
    median = np.median(s)

    abs_dev =[]
    for x in s:
        ad = abs(x - median) 
        abs_dev.append(ad)
        
    mad = np.median(abs_dev)
    thrs = mad*1.5
    
    dnm=[]
    less_contours2 = []
    for cnt2 in contours2:
      if cv.contourArea(cnt2) >= mad:
          less_contours2.append(cnt2)
          dnm.append(cv.contourArea(cnt2))

    centroids_org = []
    for cnt_o in less_contours1:
      if cv.contourArea(cnt_o) > 0 :  
        M_1 = cv.moments(cnt_o)
        cX_1 = int(M_1["m10"] / M_1["m00"])
        cY_1 = int(M_1["m01"] / M_1["m00"])
        centroids_org.append((cX_1, cY_1))

    centroids_pred = []
    for cnt_p in less_contours2:
      if cv.contourArea(cnt_p) > 0 :  
        M_2 = cv.moments(cnt_p)
        cX_2 = int(M_2["m10"] / M_2["m00"])
        cY_2 = int(M_2["m01"] / M_2["m00"])
        centroids_pred.append((cX_2, cY_2))

    x1 = []
    y1 = []

    for i in centroids_org:
      x1.append(i[0])
      y1.append(i[1])

    data_org = np.array((x1,y1))

    x2 = []
    y2 = []

    for i in centroids_pred:
      x2.append(i[0])
      y2.append(i[1])

    data_pred = np.array((x2,y2))

    tck1,u1 = interpolate.splprep(data_org, s=100, k=3)
    unew1 = np.arange(0, 1.01, 0.001)
    out1 = interpolate.splev(unew1, tck1)

    tck2,u2 = interpolate.splprep(data_pred, s=100, k=3)
    unew2 = np.arange(0, 1.01, 0.001)
    out2 = interpolate.splev(unew2, tck2)
   
    d_org= []
    for a1, b1 in zip(out1[0], out1[1]):
        st_o = int(a1)
        en_o = int(b1)
        d_org.append([st_o,en_o])

    d_pred= []
    for a2, b2 in zip(out2[0], out2[1]):
        st_p = int(a2)
        en_p = int(b2)
        d_pred.append([st_p,en_p])
          
    thickness =15
    color = (0, 0, 255)     
    for i in range(len(d_org) - 1):
        start_point1 = d_org[i]
        end_point1 = d_org[i + 1]

        cv.line(im_label, start_point1, end_point1, color, thickness)
        cv.line(spine, start_point1, end_point1, color, thickness)

    color = (255, 0, 0)     
    for i in range(len(d_pred) - 1):
        start_point2 = d_pred[i]
        end_point2 = d_pred[i + 1]

        base1, ext1 = os.path.splitext(os.path.basename(label))  
        ext1 = ext1.lower()  
        base2, ext2 = os.path.splitext(os.path.basename(prediction))  
        ext2 = ext2.lower()  

        cv.line(spine_pred, start_point2, end_point2, color, thickness)

    pixel_spacing = [0.179363, 0.179363]

    differences = []
    
    for i in range(min(len(d_pred), len(d_org))):  
        try:
            surg_x, surg_y = d_pred[i]
            pred_x, pred_y = d_org[i]
    
            diff_x = pred_x - surg_x
            diff_y = pred_y - surg_y
    
            diff_x_mm = diff_x * pixel_spacing[0]
            diff_y_mm = diff_y * pixel_spacing[1]
            differences.append((diff_x_mm, diff_y_mm))
        except IndexError: 
            pass  
    
        MSE = np.square(differences).mean()
        RMSE = math.sqrt(MSE)
    a.append(s)
    c.append(k)
    print(base1)
    print(RMSE)
    destination_path = os.path.join("./output/rmse", f"{base1}.txt")

    f = open(destination_path, "w")
    f.write(str(RMSE))
    f.close()

    cv.imwrite(f"./output/spline/{base1}{ext1}", spine)
    cv.imwrite(f"./output/spline/{base2}{ext2}", spine_pred)
    

In [ ]:
print(RMSE)

In [ ]:
#predictions
y = []

for i in range(len(a)):
    t = a[i]
    for i in range(len(t)):
        r = int(t[i])
        y.append(r)


#original images
p = []

for i in range(len(c)):
    g = c[i]
    for i in range(len(g)):
        l = int(g[i])
        p.append(l)


In [ ]:
print(statistics.harmonic_mean(s))
k

In [ ]:
#predictions
print(min(y), max(y))

In [ ]:
#org
print(min(p), max(p))